In [1]:
import pandas as pd 
import numpy as np 
import requests
import json
from tqdm import tqdm
import random

In [2]:
cassandra = pd.read_csv('lang_game.csv')

In [3]:
cassandra['language_code'].value_counts()

EN    728602
FR    728602
RU    728602
Name: language_code, dtype: int64

In [4]:
mongo = pd.read_json('static_data.json')

In [5]:
mongo = mongo.rename({'game_id': 'gameId'},axis=1)
cassandra = cassandra.rename({'game_id': 'gameId'},axis=1)
mongo = mongo.rename({'release_date': 'releaseDate'},axis=1)

In [6]:
start_date = pd.Timestamp('2006-01-01')
end_date = pd.Timestamp('2024-03-31')
num_dates = len(mongo)
mongo['releaseDate'] = [pd.Timestamp(random.randint(start_date.value, end_date.value)) for _ in range(num_dates)]



# преобразуем столбец в нужный формат
mongo['releaseDate'] = mongo['releaseDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f') 
mongo['releaseDate'] = mongo['releaseDate'].apply(lambda x: x[:-3]) + 'Z'

In [7]:
mongo.head(1)

,gameId,developer,publisher,releaseDate,genre,platform,rating
0,0,Matt Cohen,Tonguç Bodur,2021-12-28T23:00:18.788Z,"Violent,Gore,Action,Indie,RPG,Early Access",PS,8


In [8]:
mongo_json = mongo.to_dict(orient='records')

In [9]:
def make_dict_from_dfrow(df):
    """ Ф-ция создает countryInfos dict"""
    result = {}
    for index, row in df.iterrows():
        dict_result = {}
        language_code = row['language_code']
        title = row['titel']
        description = row['decription']
        price = row['price']
        dict_result[language_code] = {'title': title, 'description': description, 'price': price}
        result.update(dict_result)
    return result

In [10]:
result = []
for record in tqdm(mongo_json):
    commonInfo = {'commonInfo' : record}
    
    cassandra_id = cassandra[cassandra['gameId'] == record['gameId']]
    countryInfos = make_dict_from_dfrow(cassandra_id)
    countryInfos = {'countryInfos' : countryInfos}
    
    result_record = {
    'commonInfo': commonInfo['commonInfo'],
    'countryInfos': countryInfos['countryInfos']
     }
    result.append(result_record)


100%|█████████████████████████████████████████████████████████████████████████| 728602/728602 [59:07<00:00, 205.36it/s]


In [14]:
import pickle
with open('result.pkl', 'wb') as file:
    pickle.dump(result, file)

In [15]:
len(result)

728602

# Main upload pipeline

In [1]:
import pickle

In [2]:
with open('result.pkl', 'rb') as file:
    result = pickle.load(file)

In [2]:
url = 'http://localhost:8080/game-shop/add-game/'

headers = {'Content-Type': 'application/json', 'accept': '*/*'}
try:
    for record in tqdm(result):
        json_data = json.dumps(record)
        response = requests.post(url, data=json_data, headers=headers)
except Exception as e:
    print(e)
    print(response.text)

# Thread

In [ ]:
import pickle
import json
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

with open('result.pkl', 'rb') as file:
    result = pickle.load(file)

url = 'http://localhost:8080/game-shop/add-game/'

headers = {'Content-Type': 'application/json', 'accept': '*/*'}

def send_request(record):
    json_data = json.dumps(record)
    response = requests.post(url, data=json_data, headers=headers)
    return response

pool = ThreadPoolExecutor(max_workers=5)

for page in tqdm(pool.map(send_request, result)):
    try:
        page
    except Exception as e:
        print(e)


88637it [21:03, 56.75it/s] 